In [237]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from functools import reduce
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import xlsxwriter

In [102]:
df = pd.read_csv('Data Set 2.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533327 entries, 0 to 533326
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           533327 non-null  int64 
 1   customer_id  533327 non-null  int64 
 2   location_id  533327 non-null  int64 
 3   channel      533327 non-null  object
 4   amount       533327 non-null  object
 5   bill_no      533327 non-null  object
 6   created_on   533327 non-null  object
dtypes: int64(3), object(4)
memory usage: 28.5+ MB


In [103]:
df['billed_date'] = pd.to_datetime(df['created_on'], infer_datetime_format=True)
df.drop('created_on',axis=1,inplace=True)
df

,id,customer_id,location_id,channel,amount,bill_no,billed_date
0,42877344,60,192,uber,1039,GGN001-24323,2018-06-24 00:00:00
1,35625005,134,535,takeaway,1029,DEL007-1522,2018-01-14 19:21:00
2,44397299,166,347,uber,355,DEL005-29173,2018-09-13 13:40:00
3,44405460,166,347,uber,371,DEL005-29180,2018-09-13 14:33:00
4,44717004,166,347,uber,371,DEL005-29603,2018-09-16 12:40:00
...,...,...,...,...,...,...,...
533322,63920571,11655184,218,zomato,674,GGN004-49266,2019-10-04 14:21:00
533323,63920575,11655185,4002,dine in,1565,DEL009-26660,2019-10-04 10:11:00
533324,63920583,11655186,194,zomato,217,GGN002-70779,2019-10-04 15:44:00
533325,63920586,11655187,192,uber,252,GGN001-88302,2019-10-04 16:21:00


In [104]:
df['month'] = df['billed_date'].values.astype('datetime64[M]')
df_formatted  = df[df.amount.apply(lambda x: x.isnumeric())]

In [105]:
df_formatted['amount'] = pd.to_numeric(df_formatted['amount'])

In [138]:
mom_orders = df_formatted.groupby('month').size().reset_index(name='orders')
mom_orders

,month,counts
0,2018-01-01,28845
1,2018-02-01,27427
2,2018-03-01,25517
3,2018-04-01,27057
4,2018-05-01,27904
5,2018-06-01,32083
6,2018-07-01,33454
7,2018-08-01,28313
8,2018-09-01,37353
9,2018-10-01,35939


In [168]:
mom_revenee_unqcust = df_formatted.groupby("month").agg({"amount": np.sum, "customer_id": lambda x: x.nunique()})
mom_revenee_unqcust.rename(columns={'customer_id': 'unique_customers'}, inplace=True)
mom_revenee_unqcust

,amount,unique_customers
month,,
2018-01-01,17210945,22310
2018-02-01,16681065,21588
2018-03-01,15052840,19192
2018-04-01,15582275,20542
2018-05-01,16251829,21165
2018-06-01,18717523,24788
2018-07-01,19673116,25989
2018-08-01,16533437,22499
2018-09-01,18418843,28357


In [146]:
df_formatted['cohort'] = df_formatted.groupby('customer_id')['billed_date'].transform('min').dt.to_period('M')
df_formatted

,id,customer_id,location_id,channel,amount,bill_no,billed_date,month,cohort
0,42877344,60,192,uber,1039,GGN001-24323,2018-06-24 00:00:00,2018-06-01,2018-06
1,35625005,134,535,takeaway,1029,DEL007-1522,2018-01-14 19:21:00,2018-01-01,2018-01
2,44397299,166,347,uber,355,DEL005-29173,2018-09-13 13:40:00,2018-09-01,2018-09
3,44405460,166,347,uber,371,DEL005-29180,2018-09-13 14:33:00,2018-09-01,2018-09
4,44717004,166,347,uber,371,DEL005-29603,2018-09-16 12:40:00,2018-09-01,2018-09
...,...,...,...,...,...,...,...,...,...
533322,63920571,11655184,218,zomato,674,GGN004-49266,2019-10-04 14:21:00,2019-10-01,2019-10
533323,63920575,11655185,4002,dine in,1565,DEL009-26660,2019-10-04 10:11:00,2019-10-01,2019-10
533324,63920583,11655186,194,zomato,217,GGN002-70779,2019-10-04 15:44:00,2019-10-01,2019-10
533325,63920586,11655187,192,uber,252,GGN001-88302,2019-10-04 16:21:00,2019-10-01,2019-10


In [199]:
mom_first_order_cust_sort = df_formatted.sort_values('billed_date')
mom_first_order_cust = mom_first_order_cust_sort.assign(Occurence=np.where(~mom_first_order_cust_sort['customer_id'].duplicated(),'New','Existing'))
mom_first_order = mom_first_order_cust[mom_first_order_cust['Occurence'] == 'New'].groupby(['month']).size().reset_index(name='New_Customers')
mom_first_order

,month,New_Customers
0,2018-01-01,22310
1,2018-02-01,17509
2,2018-03-01,13435
3,2018-04-01,13316
4,2018-05-01,13482
5,2018-06-01,15624
6,2018-07-01,15920
7,2018-08-01,12595
8,2018-09-01,16851
9,2018-10-01,15079


In [159]:
mom_freq = df_formatted.groupby(['month']).apply(lambda x: x['amount'].sum()/len(x)).reset_index(name='frequency')
mom_freq

,month,frequency
0,2018-01-01,596.669960
1,2018-02-01,608.198673
2,2018-03-01,589.914175
3,2018-04-01,575.905496
4,2018-05-01,582.419331
5,2018-06-01,583.409376
6,2018-07-01,588.064686
7,2018-08-01,583.952142
8,2018-09-01,493.102107
9,2018-10-01,594.979048


In [204]:
mom_exsisting =  mom_first_order_cust[mom_first_order_cust['Occurence'] == 'Existing'].groupby(['month']).size().reset_index(name='Exsisting_Customers')
mom_exsisting

,month,Exsisting_Customers
0,2018-01-01,6535
1,2018-02-01,9918
2,2018-03-01,12082
3,2018-04-01,13741
4,2018-05-01,14422
5,2018-06-01,16459
6,2018-07-01,17534
7,2018-08-01,15718
8,2018-09-01,20502
9,2018-10-01,20860


In [258]:
df_test = [mom_revenee_unqcust,mom_first_order,mom_orders,mom_freq,mom_exsisting]
df_overall = reduce(lambda  left,right: pd.merge(left,right,on=['month'],
                                            how='inner'), df_test) 
df_overall.rename(columns={'counts':'orders','amount':'revenue'},inplace=True)
df_overall_final = df_overall.transpose()

In [259]:
list_loc = pd.unique(df_formatted['location_id'])
list_loc

array([ 192,  535,  347, 4593,  270,  272, 4594, 4416, 4002,  218, 4271,
        295,  346,  193,  271,  194, 4003, 5097, 5065, 4856, 5063, 4857,
       5093,  269, 5064, 4929, 5098, 5092, 5096, 5062, 5073, 5099, 5095],
      dtype=int64)

In [260]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('MOM_Report.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df_overall_final.to_excel(writer, sheet_name='Overall')

count = 0 # POC purpose
for i in list_loc:
    count = count + 1
    mom_loc_orders = df_formatted.groupby(['month','location_id']).size().reset_index(name='orders')
    mom_loc_orders.loc[mom_loc_orders['location_id'] == i]

    mom_loc_revenee_unqcust = df_formatted.groupby(["month",'location_id'],as_index=False).agg({"amount": np.sum, "customer_id": lambda x: x.nunique()})
    mom_loc_revenee_unqcust.rename(columns={'customer_id': 'unique_customers'}, inplace=True)
    mom_loc_revenee_unqcust.loc[mom_loc_revenee_unqcust['location_id'] == i]

    mom_loc_freq = df_formatted.groupby(['month','location_id']).apply(lambda x: x['amount'].sum()/len(x)).reset_index(name='frequency')
    mom_loc_freq.loc[mom_loc_freq['location_id']==i]
    
    mom_loc_first_order_cust_sort = df_formatted.sort_values('billed_date')
    mom_loc_first_order_cust = mom_loc_first_order_cust_sort.assign(Occurence=np.where(~mom_loc_first_order_cust_sort['customer_id'].duplicated(),'New','Existing'))
    mom_loc_first_order = mom_loc_first_order_cust[mom_loc_first_order_cust['Occurence'] == 'New'].groupby(['month','location_id']).size().reset_index(name='New_Customers')
    mom_loc_first_order.loc[mom_loc_first_order['location_id'] == i]
    
    mom_exsisting_loc = mom_loc_first_order_cust[mom_loc_first_order_cust['Occurence'] == 'Existing'].groupby(['month','location_id']).size().reset_index(name='Exsisting_Customers')
    mom_exsisting_loc.loc[mom_exsisting_loc['location_id'] == i]
    
    df_test_loc = [mom_loc_orders,mom_loc_revenee_unqcust,mom_loc_freq,mom_loc_first_order,mom_exsisting_loc]
    df_loc = reduce(lambda  left,right: pd.merge(left,right,on=['month','location_id'],
                                            how='inner'), df_test_loc) 
    df_loc.rename(columns={'counts':'orders','amount':'revenue'},inplace=True)
    df_loc = df_loc.transpose()
    
    name= 'Loc_'+str(i)
    df_loc.to_excel(writer, sheet_name=name)
    
    if count == 6:
        break

writer.save()
writer.close()

## POC MAIL

In [ ]:
fromaddr = "xxxxxxxxxx"
toaddr = "xxxxxxxxxx"
   
# instance of MIMEMultipart
msg = MIMEMultipart()
  
# storing the senders email address  
msg['From'] = fromaddr
  
# storing the receivers email address 
msg['To'] = toaddr
  
# storing the subject 
msg['Subject'] = "MOM Sales Report"
  
# string to store the body of the mail
body = "Please find attached MOM Sales report"
  
# attach the body with the msg instance
msg.attach(MIMEText(body, 'plain'))
  
# open the file to be sent 
filename = "MOM_Report.xlsx"
attachment = open("MOM_Report.xlsx", "rb")
  
# instance of MIMEBase and named as p
p = MIMEBase('application', 'octet-stream')
  
# To change the payload into encoded form
p.set_payload((attachment).read())
  
# encode into base64
encoders.encode_base64(p)
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
  
# attach the instance 'p' to instance 'msg'
msg.attach(p)
  
# creates SMTP session
s = smtplib.SMTP('smtp.gmail.com', 587)
  
# start TLS for security
s.starttls()
  
# Authentication
s.login(fromaddr, "XXXXXXXX")
  
# Converts the Multipart msg into a string
text = msg.as_string()
  
# sending the mail
s.sendmail(fromaddr, toaddr, text)
  
# terminating the session
s.quit()